In [2]:
import pandas as pd
import numpy as np
import time
import pyarrow.parquet as pq
import scipy
#import implicit
import bisect
import sklearn.metrics as m
#from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

In [3]:
#чтение, вывод инфы
data = pq.read_table('data/smoking.parquet')
display(pd.DataFrame([(z.name, z.type) for z in data.schema], columns = [['field', 'type']]))

targets = pq.read_table('data/target.parquet')
display(pd.DataFrame([(z.name, z.type) for z in data.schema], columns = [['field', 'type']]))

,field,type
0,ID,int64
1,gender,string
2,age,int64
3,height(cm),int64
4,weight(kg),int64
5,waist(cm),double
6,eyesight(left),double
7,eyesight(right),double
8,hearing(left),double
9,hearing(right),double


In [4]:
data.select(['gender']).to_pandas()['gender'].value_counts()

M    35401
F    20291
Name: gender, dtype: int64

In [ ]:
%%time
data_agg = data.select(['user_id', 'url_host', 'request_cnt']).group_by(['user_id', 'url_host']).aggregate([('request_cnt', "sum")])

In [ ]:
url_set = set(data_agg.select(['url_host']).to_pandas()['url_host'])
print(f'{len(url_set)} urls')
url_dict = {url: idurl for url, idurl in zip(url_set, range(len(url_set)))}
usr_set = set(data_agg.select(['user_id']).to_pandas()['user_id'])
print(f'{len(usr_set)} users')
usr_dict = {usr: user_id for usr, user_id in zip(usr_set, range(len(usr_set)))}

Делаем странные вещи

In [ ]:
%%time
values = np.array(data_agg.select(['request_cnt_sum']).to_pandas()['request_cnt_sum'])
rows = np.array(data_agg.select(['user_id']).to_pandas()['user_id'].map(usr_dict))
cols = np.array(data_agg.select(['url_host']).to_pandas()['url_host'].map(url_dict))
mat = scipy.sparse.coo_matrix((values, (rows, cols)), shape=(rows.max() + 1, cols.max() + 1))
als = implicit.approximate_als.FaissAlternatingLeastSquares(factors = 50, iterations = 30, use_gpu = False, calculate_training_loss = False, regularization = 0.1)

In [ ]:
%%time
als.fit(mat)
u_factors = als.model.user_factors
d_factors = als.model.item_factors

типа подготовка данных

In [ ]:
%%time
inv_usr_map = {v: k for k, v in usr_dict.items()}
usr_emb = pd.DataFrame(u_factors)
usr_emb['user_id'] = usr_emb.index.map(inv_usr_map)
usr_targets = targets.to_pandas()
df = usr_targets.merge(usr_emb, how = 'inner', on = ['user_id'])
df = df[df['is_male'] != 'NA']
df = df.dropna()
df['is_male'] = df['is_male'].map(int)
df['is_male'].value_counts()

каким-то боком еще и кетбуст

In [ ]:
%%time
x_train, x_test, y_train, y_test = train_test_split( \
    df.drop(['user_id', 'age', 'is_male'], axis = 1), df['is_male'], test_size = 0.33, random_state = SPLIT_SEED)
clf = CatBoostClassifier()
clf.fit(x_train, y_train, verbose = False)
print(f'GINI по полу {2 * m.roc_auc_score(y_test, clf.predict_proba(x_test)[:,1]) - 1:2.3f}')

С возрастом у них более менее адекватно, есть крутая штука classification_report